In [4]:
from pytorch_fid import fid_score

import os
from skimage.metrics import structural_similarity as ssim
from skimage import io
from skimage.color import rgb2gray
import numpy as np
import cv2
import numpy as np
import os
from math import log10, sqrt

In [5]:
import tqdm 
import pandas as pd 
from pytorch_fid.fid_score import InceptionV3, calculate_frechet_distance, compute_statistics_of_path
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
from PIL import Image

def get_img_metric(img_dir, img_w_dir, num_imgs=None):
    filenames = os.listdir(img_dir)
    filenames.sort()
    if num_imgs is not None:
        filenames = filenames[:num_imgs]
    log_stats = []
    for ii, filename in enumerate(tqdm.tqdm(filenames)):
        pil_img_ori = Image.open(os.path.join(img_dir, filename)).convert('RGB')
        pil_img_ori = pil_img_ori.resize((512, 512), Image.BICUBIC)
        
        pil_img = Image.open(os.path.join(img_w_dir, filename)).convert('RGB')
        pil_img = pil_img.resize((512, 512), Image.BICUBIC)
        
        img_ori = np.asarray(pil_img_ori)
        img = np.asarray(pil_img)
        #print(img_ori.shape, img.shape)
        log_stat = {
            'filename': filename,
            'ssim': structural_similarity(img_ori, img, channel_axis=2),
            'psnr': peak_signal_noise_ratio(img_ori, img),
            'linf': np.amax(np.abs(img_ori.astype(int)-img.astype(int)))
        }
        log_stats.append(log_stat)
    return log_stats

In [6]:
# img_dir= "ckpts/0/test/original_imgs"
# img_w_dir = "ckpts/0/test/watermarked_imgs"
#params.num_imgs = 10000

img_dir= "ckpts/1_test/test/generate_imgs"
img_w_dir = "ckpts/1_test/test/generate_w_imgs"
img_metrics = get_img_metric(img_dir, img_w_dir)
img_df = pd.DataFrame(img_metrics)

100%|██████████| 4224/4224 [12:30<00:00,  5.63it/s]


In [8]:
output_dir = 'ckpts/1_test/test'
os.makedirs(output_dir,exist_ok=True)
img_df.to_csv(os.path.join(output_dir, 'img_metrics2.csv'), index=False)
ssims = img_df['ssim'].tolist()
psnrs = img_df['psnr'].tolist()
linfs = img_df['linf'].tolist()
ssim_mean, ssim_std, ssim_max, ssim_min = np.mean(ssims), np.std(ssims), np.max(ssims), np.min(ssims) 
psnr_mean, psnr_std, psnr_max, psnr_min = np.mean(psnrs), np.std(psnrs), np.max(psnrs), np.min(psnrs)
linf_mean, linf_std, linf_max, linf_min = np.mean(linfs), np.std(linfs), np.max(linfs), np.min(linfs)
print(f"SSIM: {ssim_mean:.4f}±{ssim_std:.4f} [{ssim_min:.4f}, {ssim_max:.4f}]")
print(f"PSNR: {psnr_mean:.4f}±{psnr_std:.4f} [{psnr_min:.4f}, {psnr_max:.4f}]")
print(f"Linf: {linf_mean:.4f}±{linf_std:.4f} [{linf_min:.4f}, {linf_max:.4f}]")

SSIM: 0.2224±0.0957 [0.0469, 0.6810]
PSNR: 13.1282±1.8043 [8.9456, 22.9019]
Linf: 251.2644±6.4657 [198.0000, 255.0000]


In [4]:
from pytorch_fid.fid_score import InceptionV3, calculate_frechet_distance, compute_statistics_of_path,calculate_fid_given_paths

img_dir= "ckpts/1_test/test/generate_imgs"
img_w_dir = "ckpts/1_test/test/generate_w_imgs"
fid= calculate_fid_given_paths([img_w_dir,"ckpts/1_test/test/original_imgs" ],batch_size=32,device='cuda',dims=2048)
print(fid)

100%|██████████| 6/6 [00:02<00:00,  2.23it/s]


226.1023163228521


In [4]:
output_dir = 'ckpts/1_test/test'
os.makedirs(output_dir,exist_ok=True)
img_df.to_csv(os.path.join(output_dir, 'img_metrics2.csv'), index=False)
ssims = img_df['ssim'].tolist()
psnrs = img_df['psnr'].tolist()
linfs = img_df['linf'].tolist()
ssim_mean, ssim_std, ssim_max, ssim_min = np.mean(ssims), np.std(ssims), np.max(ssims), np.min(ssims) 
psnr_mean, psnr_std, psnr_max, psnr_min = np.mean(psnrs), np.std(psnrs), np.max(psnrs), np.min(psnrs)
linf_mean, linf_std, linf_max, linf_min = np.mean(linfs), np.std(linfs), np.max(linfs), np.min(linfs)
print(f"SSIM: {ssim_mean:.4f}±{ssim_std:.4f} [{ssim_min:.4f}, {ssim_max:.4f}]")
print(f"PSNR: {psnr_mean:.4f}±{psnr_std:.4f} [{psnr_min:.4f}, {psnr_max:.4f}]")
print(f"Linf: {linf_mean:.4f}±{linf_std:.4f} [{linf_min:.4f}, {linf_max:.4f}]")

SSIM: 0.2224±0.0957 [0.0469, 0.6810]
PSNR: 13.1282±1.8043 [8.9456, 22.9019]
Linf: 251.2644±6.4657 [198.0000, 255.0000]


In [5]:
img_dir= "ckpts/1_test/test/original_imgs"
img_w_dir = "ckpts/1_test/test/watermarked_imgs"

img_metrics = get_img_metric(img_dir, img_w_dir)
img_df = pd.DataFrame(img_metrics)

output_dir = 'ckpts/1_test/test'
os.makedirs(output_dir,exist_ok=True)
img_df.to_csv(os.path.join(output_dir, 'img_metrics.csv'), index=False)
ssims = img_df['ssim'].tolist()
psnrs = img_df['psnr'].tolist()
linfs = img_df['linf'].tolist()
ssim_mean, ssim_std, ssim_max, ssim_min = np.mean(ssims), np.std(ssims), np.max(ssims), np.min(ssims) 
psnr_mean, psnr_std, psnr_max, psnr_min = np.mean(psnrs), np.std(psnrs), np.max(psnrs), np.min(psnrs)
linf_mean, linf_std, linf_max, linf_min = np.mean(linfs), np.std(linfs), np.max(linfs), np.min(linfs)
print(f"SSIM: {ssim_mean:.4f}±{ssim_std:.4f} [{ssim_min:.4f}, {ssim_max:.4f}]")
print(f"PSNR: {psnr_mean:.4f}±{psnr_std:.4f} [{psnr_min:.4f}, {psnr_max:.4f}]")
print(f"Linf: {linf_mean:.4f}±{linf_std:.4f} [{linf_min:.4f}, {linf_max:.4f}]")

100%|██████████| 192/192 [00:26<00:00,  7.16it/s]

SSIM: 1.0000±0.0000 [1.0000, 1.0000]
PSNR: 75.4929±4.0502 [60.9556, 95.0462]
Linf: 1.0000±0.0000 [1.0000, 1.0000]
